In [65]:
import seqio
import os
os.environ['CURL_CA_BUNDLE'] = "/etc/ssl/certs/ca-bundle.crt"
from bigbench.bbseqio import tasks
vocabulary=seqio.SentencePieceVocabulary("/gscratch/zlab/bparan/projects/cascades/models/t5-spiece.model")
from sklearn.metrics import accuracy_score
import json
import tqdm

In [1]:
import datasets
import os
import openai
import numpy as np
with open(os.path.expanduser('~/.openai_api_key'), 'r') as file:
    openai.api_key = file.read().replace('\n', '')

import adatest

2022-10-03 12:33:32.201508: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-03 12:34:17.117214: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-10-03 12:34:17.117236: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [322]:
d = datasets.load_dataset('bigbench', 'anachronisms', cache_dir='/gscratch/zlab/bparan/projects/cascades/data')

  0%|          | 0/3 [00:00<?, ?it/s]

In [337]:
d_val = [ex['inputs'].split("\n")[0] for ex in d['validation']]
d_train = [ex['inputs'].split("\n")[0] for ex in d['train']]
# all([sent in d_train for sent in d_val])

184.0

In [141]:
import numpy as np
hg_inputs = d['validation']['inputs']
hg_inputs = [x.split('\n')[0] for x in hg_inputs]
hg_labels = np.array([int(x[0] == 'Yes') for x in d['validation']['targets']])

In [335]:
task = seqio.get_mixture_or_task(
    "bigbench:anachronisms.mul.t5_default_vocab.0_shot.all_examples")
sequence_length = {"inputs": 1024, "targets": 1024}
ds = task.get_dataset(sequence_length)

In [136]:
input_set = set()
for ex in ds:
    inp = ex['inputs_pretokenized'].numpy().decode()
    label = ex['answers'][0].numpy().decode()
    if (inp,label) in input_set:
        continue
    else:
        input_set.add((inp, label))
inputs = []
label_strings = []
for tup in input_set:
    inputs.append(tup[0].split("\n")[0])
    label_strings.append(tup[1])
labels = np.array([int(x == 'Yes') for x in label_strings])

In [8]:
class OpenAIModel(adatest.Model):
    def __init__(self, model="text-davinci-002", quote="\"", temperature=0.7, top_p=1, max_length=30, n=1):
        self.model = model
        self.api_key = openai.api_key
        self.quote = quote
        self.temperature = temperature
        self.top_p = top_p
        self.max_length = max_length
        self.n = n
    def __call__(self, strings):
        resp = openai.Completion.create(
            model=self.model,
            prompt=strings,
            max_tokens=self.max_length,
            temperature=self.temperature,
            top_p=self.top_p,
            n=self.n,
            stop=self.quote,
        )
        return [x["text"] for x in resp['choices']]

gpt3 = OpenAIModel(model="text-davinci-002",  max_length=200, quote='', n=1)


In [9]:
def anachronism(x):
    gpt3 = OpenAIModel(model="text-davinci-002",  max_length=200, quote='---', n=1)
    prompt = '''Given a sentence and the time periods of each entity in it, tell me if it could have happened or not.
Sentence: I wrote about shakespeare
Entities and dates:
I -> 21st century
Shakespeare -> 16th century
Could the sentence be true based on the dates alone: Yes
----
Sentence: Shakespeare wrote about me

Entities and dates:
Shakespeare -> 16th century
I -> 21st century

Could the sentence be true based on the dates alone: No
----
Sentence: %s''' % x
    return gpt3(prompt)


In [10]:
def anachronismv2(x):
    gpt3 = OpenAIModel(model="text-davinci-002",  max_length=200, quote='---', n=1)
    prompt = '''Given a sentence tell me if it could have happened or not based on the time periods of the entities.
Sentence: I wrote about shakespeare
Could the sentence be true based on the time periods alone: Yes
----
Sentence: Shakespeare wrote about me
Could the sentence be true based on the time periods alone: No
----
Sentence: %s''' % x
    return gpt3(prompt)

In [11]:
def anachronismv3(x):
    prompt = '''Given a sentence, extract the entities and their relationships, and tell me if they are anachronistic.
Sentence: I wrote about shakespeare
Relationships:
- (I, write about, shakespeare) -> not anachronistic, 
----
Sentence: Shakespeare wrote about me
Relationships:
- (shakespeare, write about, me) -> anachronistic
----
Sentence: The builders of the pyramids at Giza listened to jazz during their break.
Relationships:
- (builders, listen to, jazz) -> anachronistic
----
Sentence: %s
''' % x
    return gpt3(prompt)

In [524]:
perf = []
for iter in range(1):
    answers = []
    answers_without_search = []
    for x in tqdm.tqdm(inputs):
        answer = anachronism(x)
        answers.append(answer)
        answers_without_search.append(answer)
    preds = np.array([int(x[0].endswith('No')) for x in answers])
    perf.append((preds == labels).mean())
np.mean(perf)
np.std(perf)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 184/184 [06:06<00:00,  1.99s/it]


0.0

In [515]:
np.mean(perf)

0.7880434782608695

In [13]:
answers2 = []
for x in inputs:
    answers2.append(anachronismv2(x))

In [14]:

answers3 = []
for x in inputs:
    answers3.append(anachronismv3(x))

In [15]:
preds = np.array([int(x[0].endswith('No')) for x in answers])

In [16]:
preds2 = np.array([int(x[0].endswith('No')) for x in answers2])

In [17]:
preds3 = np.array([1 - int('not anachronistic' in x[0]) for x in answers3])

In [44]:
(preds == labels).mean()

0.7391304347826086

In [19]:
(preds2 == labels).mean()

0.6086956521739131

In [20]:
(preds3 == labels).mean()

0.6739130434782609

In [67]:
for x in np.where(preds != labels)[0]:
    print(inputs[x])
    print(answers[x][0])
    print(labels[x])
    print('----')

Jason, the first person to explore the ancient pyramid in over 2000 years, opened a sealed box inside and found a floppy disk.


Jason -> 21st century
ancient pyramid -> ?

Could the sentence be true based on the dates alone:

The sentence could be true if the ancient pyramid is from the 21st century.
1
----
The first ever movie in color depicted the life of Sacagawea.


Entities and dates:
The first ever movie in color -> late 19th century
Sacagawea -> late 18th century - early 19th century

Could the sentence be true based on the dates alone: No
0
----
Jason, while exploring an ancient pyramid, opened a sealed box inside and found a floppy disk.


Entities and dates:
Jason -> 21st century
floppy disk -> 1970s-2000s

Could the sentence be true based on the dates alone: No
0
----
During their meetings in Bali, George Washington and the delegate of the Tokugawa shogunate exchanged gifts.


Entities and dates:
George Washington -> 18th century
Tokugawa shogunate -> 17th century

Could th

### Web search affordance 

In [517]:
from serpapi import GoogleSearch

def search_entity(prompt_entity):
    params = {
      "q": "What time period does %s belong to? When was it created or born?" % prompt_entity,
    #   "location": "Austin, Texas, United States",
      "hl": "en",
      "gl": "us",
      "google_domain": "google.com",
      "api_key": "52a45ebae7c1189a57cd442515feebd0ee4f6ef0d02552afe4f7efa4bc73fc95"
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    return results


# Gather organic results for search
def render_results(data: dict, top_k=2) -> str:
    if not data or not data.get("organic_results"):
        return "No results found"

    results = []
    for result in data["organic_results"][:top_k]:
        title = result.get("title")
        link = result.get("link")
        snippet = result.get("snippet")
        if not title or not link or not snippet:
            continue
        results.append(f"{snippet.strip()}")

    return "\n".join(results)


def findEntities(x):
    gpt3 = OpenAIModel(model="text-davinci-002",  max_length=200, quote='---', n=1)
    prompt = '''Given a sentence, give me the entities mentioned in it.
Sentence: I wrote about shakespeare
Entities:
I
Shakespeare
----
Sentence: The Dodo was capable of hunting large herbivores like the Iguandodon
Entities:
Dodo
Iguandodon
----
Sentence: The engineer upgraded the USS New Ironsides' radar system.
Entities: 
engineer
USS New Ironsides
radar
----
Sentence: %s\nEntities:\n''' % x
    return gpt3(prompt)


def anachronismWithSearch(x, b):
    gpt3 = OpenAIModel(model="text-davinci-002",  max_length=200, quote='---', n=1)
    prompt = '''Given a sentence and backgroud information about it, tell me if it could have happened or not.
Sentence: I wrote about shakespeare

Background:
Shakespeare produced most of his known works between 1589 and 1613. ... His early plays were primarily comedies and histories and are regarded as some of the best ...
Shakespeare lived from 1567 to 1616. Scholars and historians often refer to him being a part of the Elizabethan Era, the period of English ...
Generation Z Characteristics. Gen Z, also known as iGen, Centennials, etc., starts with those born in approximately 1996. The oldest members of this generation ...

Could the sentence be true based on the background: Yes
----
Sentence: The Dodo was capable of hunting large herbivores like the Iguandodon

Background:
The dodo (Raphus cucullatus) is an extinct flightless bird that was endemic to the island of Mauritius, which is east of Madagascar in the Indian Ocean.
The dodo was extinct by 1681, the Réunion solitaire by 1746, and the Rodrigues solitaire by about 1790.
While many species have been classified in the genus Iguanodon, dating from the late Jurassic Period to the early Cretaceous Period of Asia, Europe, and North ...
Iguanodon, (genus Iguanodon), large herbivorous dinosaurs found as fossils from the Late Jurassic and Early Cretaceous periods (161.2 million to 99.6 ...

Could the sentence be true based on the background: No
----
Sentence: The engineer upgraded the USS New Ironsides' radar system.

Background:
USS New Ironsides was a wooden-hulled broadside ironclad built for the United States Navy during the American Civil War. The ship spent most of her career ...
Civil War, only the U.S.S. New Ironsides was a seagoing, high-freeboard design. Her seagoing qualities and heavy battery made her uniquely valuable to the ...
The history of radar started with experiments by Heinrich Hertz in the late 19th century that showed that radio waves were reflected by metallic objects.
The principle of radar was “rediscovered” at NRL in 1930 when L.A. Hyland observed that an aircraft flying through the beam of a transmitting antenna caused a ...

Could the sentence be true based on the background: No
----
Sentence: %s

Background:
%s

Could the sentence be true based on the background:''' % (x, b)
    return prompt, gpt3(prompt)


def anachronismWithSearch1Step(x, b):
    gpt3 = OpenAIModel(model="text-davinci-002",  max_length=200, quote='---', n=1)
    prompt = '''Given a sentence and backgroud information about it, tell me if it could have happened or not.
Sentence: I wrote about shakespeare

Background:
Shakespeare lived from 1567 to 1616. Scholars and historians often refer to him being a part of the Elizabethan Era, the period of English ...
Generation Z Characteristics. Gen Z, also known as iGen, Centennials, etc., starts with those born in approximately 1996. The oldest members of this generation ...

Could the sentence be true based on the background: Yes
----
Sentence: The Dodo was capable of hunting large herbivores like the Iguandodon

Background:
The dodo was extinct by 1681, the Réunion solitaire by 1746, and the Rodrigues solitaire by about 1790.
Iguanodon, (genus Iguanodon), large herbivorous dinosaurs found as fossils from the Late Jurassic and Early Cretaceous periods (161.2 million to 99.6 million years ago)

Could the sentence be true based on the background: No
----
Sentence: The engineer upgraded the USS New Ironsides' radar system.

Background:
USS New Ironsides was a wooden-hulled broadside ironclad built for the United States Navy during the American Civil War. The ship spent most of her career ...
The history of radar started with experiments by Heinrich Hertz in the late 19th century that showed that radio waves were reflected by metallic objects.

Could the sentence be true based on the background: No
----
Sentence: %s

Background:
%s

Could the sentence be true based on the background:''' % (x, b)
    return prompt, gpt3(prompt)


def fineTimePeriod(x, b):
    gpt3 = OpenAIModel(model="text-davinci-002",  max_length=200, quote='---', n=1)
#     prompt = '''Given an entity and backgroud information about it, what century did it exist in?
# Shakespeare
# Shakespeare produced most of his known works between 1589 and 1613. ... His early plays were primarily comedies and histories and are regarded as some of the best ...
# Shakespeare lived from 1567 to 1616. Scholars and historians often refer to him being a part of the Elizabethan Era, the period of English ...
# Shakespeare -> 16th century

# I
# Generation Z Characteristics. Gen Z, also known as iGen, Centennials, etc., starts with those born in approximately 1996. The oldest members of this generation ...
# I -> 20st century

# Dodo
# The dodo (Raphus cucullatus) is an extinct flightless bird that was endemic to the island of Mauritius, which is east of Madagascar in the Indian Ocean.
# The dodo was extinct by 1681, the Réunion solitaire by 1746, and the Rodrigues solitaire by about 1790.
# Dodo -> 17th century

# radar
# The principle of radar was “rediscovered” at NRL in 1930 when L.A. Hyland observed that an aircraft flying through the beam of a transmitting antenna caused a ...
# The history of radar started with experiments by Heinrich Hertz in the late 19th century that showed that radio waves were reflected by metallic objects.
# radar -> 20th century

# %s
# %s
# ''' % (x, b)
    prompt = '''Given an entity and backgroud information about it, what time period did it exist in?
Entity: Shakespeare
Shakespeare produced most of his known works between 1589 and 1613. ... His early plays were primarily comedies and histories and are regarded as some of the best ...
Shakespeare lived from 1567 to 1616. Scholars and historians often refer to him being a part of the Elizabethan Era, the period of English ...

Shakespeare -> 1567-1616
----
Entity: I
Generation Z Characteristics. Gen Z, also known as iGen, Centennials, etc., starts with those born in approximately 1996. The oldest members of this generation ...

I -> from 1996
----
Entity: Dodo
The dodo (Raphus cucullatus) is an extinct flightless bird that was endemic to the island of Mauritius, which is east of Madagascar in the Indian Ocean.
The dodo was extinct by 1681, the Réunion solitaire by 1746, and the Rodrigues solitaire by about 1790.

Dodo -> 1681
----
Entity: Iguandodon
While many species have been classified in the genus Iguanodon, dating from the late Jurassic Period to the early Cretaceous Period of Asia, Europe, and North ...
Iguanodon, (genus Iguanodon), large herbivorous dinosaurs found as fossils from the Late Jurassic and Early Cretaceous periods (161.2 million to 99.6 ...

Iguandodon -> 99.6 million years ago
----
Entity: USS New Ironsides
USS New Ironsides was a wooden-hulled broadside ironclad built for the United States Navy during the American Civil War. The ship spent most of her career ...
Civil War, only the U.S.S. New Ironsides was a seagoing, high-freeboard design. Her seagoing qualities and heavy battery made her uniquely valuable to the ...

USS New Ironsides -> American Civil War
----
Entity: radar
The principle of radar was “rediscovered” at NRL in 1930 when L.A. Hyland observed that an aircraft flying through the beam of a transmitting antenna caused a ...
The history of radar started with experiments by Heinrich Hertz in the late 19th century that showed that radio waves were reflected by metallic objects.

radar -> 1930
----
Entity: %s
%s

''' % (x, b)
    return prompt, gpt3(prompt)
    
def anachronismWithSearchForCentury(x, b):
    gpt3 = OpenAIModel(model="text-davinci-002",  max_length=200, quote='---', n=1)
#     prompt = '''Given a sentence tell me if it could have happened or not based on the time periods of the entities.
# Sentence: I wrote about shakespeare

# Shakespeare -> 16th century
# I -> 20st century

# Could the sentence be true based on the time periods alone: Yes
# ----
# Sentence: The Dodo was capable of hunting large herbivores like the Iguandodon

# Dodo -> 17th century
# Iguanodon -> 99.6 million years ago

# Could the sentence be true based on the time periods alone: No
# ----
# Sentence: The engineer upgraded the USS New Ironsides' radar system.

# USS New Ironsides -> 19th century
# radar -> 20th century

# Could the sentence be true based on the time periods alone: No
# ----
# Sentence: %s

# %s

# Could the sentence be true based on the time periods alone:''' % (x, b)
    prompt = '''Given a sentence tell me if it could have happened or not based on the time periods of the entities.
Sentence: I wrote about shakespeare

Entities and dates:
Shakespeare -> 1567-1616
I -> from 1996

Could the sentence be true based on the time periods alone: Yes
----
Sentence: Shakespeare wrote about me

Entities and dates:
Shakespeare -> from 1996
I -> 1567-1616

Could the sentence be true based on the dates alone: No
----
Sentence: The Dodo was capable of hunting large herbivores like the Iguandodon

Entities and dates:
Dodo -> 1681
Iguanodon -> 99.6 million years ago

Could the sentence be true based on the time periods alone: No
----
Sentence: The engineer upgraded the USS New Ironsides' radar system.

Entities and dates:
USS New Ironsides -> American Civil War
radar -> 1930

Could the sentence be true based on the time periods alone: No
----
Sentence: %s

Entities and dates:
%s

Could the sentence be true based on the time periods alone:''' % (x, b)
    return prompt, gpt3(prompt)




def anachronismWithSearchCombined(x, b):
    gpt3 = OpenAIModel(model="text-davinci-002",  max_length=200, quote='---', n=1)
    prompt = '''Given a sentence, backgroud information about it, and the time periods of each entity, tell me if it could have happened or not.
Sentence: I wrote about shakespeare

Background:
Shakespeare lived from 1567 to 1616. Scholars and historians often refer to him being a part of the Elizabethan Era, the period of English ...
Generation Z Characteristics. Gen Z, also known as iGen, Centennials, etc., starts with those born in approximately 1996. The oldest members of this generation ...

Entities and time periods:
Shakespeare -> 16th century 
I -> 20th century

Could the sentence be true based on the time periods alone: Yes
----
Sentence: The Dodo was capable of hunting large herbivores like the Iguandodon

Background:
The dodo was extinct by 1681, the Réunion solitaire by 1746, and the Rodrigues solitaire by about 1790.
Iguanodon, (genus Iguanodon), large herbivorous dinosaurs found as fossils from the Late Jurassic and Early Cretaceous periods (161.2 million to 99.6 million years ago)

Entities and time periods:
dodo -> 17th century 
Iguanodon -> 99.6 million ago

Could the sentence be true based on the time periods alone: No
----
Sentence: The engineer upgraded the USS New Ironsides' radar system.

Background:
USS New Ironsides was a wooden-hulled broadside ironclad built for the United States Navy during the American Civil War. The ship spent most of her career ...
The history of radar started with experiments by Heinrich Hertz in the late 19th century that showed that radio waves were reflected by metallic objects.

Entities and time periods:
USS New Ironsides -> American Civil War
radar -> late 19th century

Could the sentence be true based on the time periods alone: No
----
Sentence: %s

Background:
%s

Entities and time periods:
''' % (x, b)
    return prompt, gpt3(prompt)

In [301]:
search_background = []
all_search_dict = []
entity_list = []
for x in tqdm.tqdm(inputs):
    entity_list.append(findEntities(x))
for ex in tqdm.tqdm(entity_list):
    ents = ex[0].split("\n")
    combined_output = ""
    search_dict = []
    for entity in ents:
        search_results = search_entity(entity)
        search_dict.append(search_results)
        search_prompt = render_results(search_results)
        combined_output += search_prompt + "\n"
    search_background.append(combined_output)
    all_search_dict.append(search_dict)

  0%|                                                                                                     | 0/184 [00:00<?, ?it/s]

https://serpapi.com/search
https://serpapi.com/search


  1%|▌                                                                                            | 1/184 [00:03<11:19,  3.71s/it]

https://serpapi.com/search
https://serpapi.com/search


  1%|█                                                                                            | 2/184 [00:09<15:14,  5.02s/it]

...

 99%|██████████████████████████████████████████████████████████████████████████████████████████ | 182/184 [18:53<00:05,  2.66s/it]

https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search


 99%|██████████████████████████████████████████████████████████████████████████████████████████▌| 183/184 [18:56<00:02,  2.91s/it]

https://serpapi.com/search
https://serpapi.com/search


100%|███████████████████████████████████████████████████████████████████████████████████████████| 184/184 [19:08<00:00,  6.24s/it]


In [317]:
all_search_dict[10][0]

{'search_metadata': {'id': '633bb20c0f9866f823973b67',
  'status': 'Success',
  'json_endpoint': 'https://serpapi.com/searches/b13874963a95fce2/633bb20c0f9866f823973b67.json',
  'created_at': '2022-10-04 04:09:48 UTC',
  'processed_at': '2022-10-04 04:09:48 UTC',
  'google_url': 'https://www.google.com/search?q=What+time+period+does+Elon+Musk+belong+to%3F+When+was+it+created+or+born%3F&oq=What+time+period+does+Elon+Musk+belong+to%3F+When+was+it+created+or+born%3F&hl=en&gl=us&sourceid=chrome&ie=UTF-8',
  'raw_html_file': 'https://serpapi.com/searches/b13874963a95fce2/633bb20c0f9866f823973b67.html',
  'total_time_taken': 1.09},
 'search_parameters': {'engine': 'google',
  'q': 'What time period does Elon Musk belong to? When was it created or born?',
  'google_domain': 'google.com',
  'hl': 'en',
  'gl': 'us',
  'device': 'desktop'},
 'search_information': {'organic_results_state': 'Results for exact spelling',
  'query_displayed': 'What time period does Elon Musk belong to? When was it 

In [500]:
search_background_1step = []
for search_dict in all_search_dict:
    combined_output = ""
    for entity_results in search_dict:
        search_prompt = render_results(entity_results, top_k=1)
        combined_output += search_prompt + "\n"
    search_background_1step.append(combined_output)
    
search_background = []
for search_dict in all_search_dict:
    combined_output = ""
    for entity_results in search_dict:
        search_prompt = render_results(entity_results, top_k=2)
        combined_output += search_prompt + "\n"
    search_background.append(combined_output)

In [523]:
perf = []
for i in range(1):
    answers = []
    answers_with_search = []
    for sentence, background in tqdm.tqdm(zip(inputs, search_background_1step)):
        prompt, answer = anachronismWithSearchCombined(sentence, background.strip())
        answers.append(answer)
        answers_with_search.append(answer)
    preds = np.array([int(x[0].endswith('No')) for x in answers])
    perf.append((preds == labels).mean())
print(np.mean(perf))
print(np.std(perf))

184it [05:21,  1.75s/it]

0.7065217391304348
0.0


In [522]:
answers

[['William the Conqueror -> 11th century \nRed Hot Chili Peppers -> 20th century\n\nCould the sentence be true based on the time periods alone: No'],
 ['woman -> any time period\nwoolly mammoth -> extinction in the Holocene epoch\n\nCould the sentence be true based on the time periods alone: No'],
 ['Pyrrhus -> 3rd century BCE \n Roman armies -> 1st century CE\n\nCould the sentence be true based on the time periods alone: No'],
 ['Homo neanderthalensis -> 40,000 years ago\ncorn -> 10,000 years ago\n\nCould the sentence be true based on the time periods alone: No'],
 ['Ivan the Terrible -> 16th century\nJohn Williams -> 20th century\n\nCould the sentence be true based on the time periods alone: No'],
 ['hoplite -> 8th century BC\nmusket -> 16th century\ncannon -> 12th century'],
 ['Genghis Khan -> 13th century \nGreat Wall of China -> 7th century BC\n\nCould the sentence be true based on the time periods alone: No'],
 ['Claude Monet -> 19th century\nRenaissance -> 15th and 16th centurie

In [285]:
perf = []
for i in range(10):
    answers = []
    for sentence, background in tqdm.tqdm(zip(inputs, search_background)):
        prompt, answer = anachronismWithSearch(sentence, background.strip())
        answers.append(answer)
    preds = np.array([int(x[0].endswith('No')) for x in answers])
    perf.append((preds == labels).mean())
np.mean(perf)
np.std(perf)

184it [01:54,  1.61it/s]
184it [01:49,  1.68it/s]
184it [01:55,  1.60it/s]
184it [01:52,  1.64it/s]
184it [01:48,  1.69it/s]
184it [01:55,  1.60it/s]
184it [01:53,  1.63it/s]
184it [01:50,  1.66it/s]
184it [01:53,  1.62it/s]
184it [01:51,  1.65it/s]


0.009094130723196474

In [286]:
np.mean(perf)

0.7065217391304348

In [295]:
len(search_background)

184

In [400]:
perf = []
for i in range(10):
    answers = []
    for sentence, background in tqdm.tqdm(zip(inputs, search_background_1step)):
        prompt, answer = anachronismWithSearch1Step(sentence, background.strip())
        answers.append(answer)
    preds = np.array([int(x[0].endswith('No')) for x in answers])
    perf.append((preds == labels).mean())
print(np.mean(perf))
print(np.std(perf))

0it [00:00, ?it/s]

0.5108695652173914
0.0


In [477]:
## Program to use search results to get better estimates of century and then use that for anachronism task.
perf = []
for i in range(5):
    retrieval_based_time_periods = []
    for i, ex in tqdm.tqdm(enumerate(entity_list)):
        ents = ex[0].split("\n")
        entity_search_results = all_search_dict[i]
        all_time_periods = ""
        for entity, search_background in zip(ents, entity_search_results):
            # Combine with the right background info , send to get get centuries
            background_text = render_results(search_background, top_k=2)
            prompt, time_period = fineTimePeriod(entity, background_text.strip())
            all_time_periods += time_period[0].strip() + "\n"
        retrieval_based_time_periods.append(all_time_periods)

    answers = []
    for sentence, background in tqdm.tqdm(zip(inputs, retrieval_based_time_periods)):
        prompt, answer = anachronismWithSearchForCentury(sentence, background.strip())
        answers.append(answer)
    preds = np.array([int(x[0].endswith('No')) for x in answers])
    perf.append((preds == labels).mean())
print(np.mean(perf))
print(np.std(perf))

184it [07:02,  2.30s/it]
184it [01:40,  1.84it/s]
184it [07:08,  2.33s/it]
184it [01:44,  1.75it/s]
184it [07:11,  2.35s/it]
184it [01:45,  1.75it/s]
184it [07:17,  2.38s/it]
184it [01:45,  1.74it/s]
184it [07:26,  2.42s/it]
184it [01:50,  1.66it/s]

0.6945652173913043
0.01513955247520016


In [473]:
print(prompt)

Given a sentence tell me if it could have happened or not based on the time periods of the entities.
Sentence: I wrote about shakespeare

Entities and dates:
Shakespeare -> 1567-1616
I -> from 1996

Could the sentence be true based on the time periods alone: Yes
----
Sentence: Shakespeare wrote about me

Entities and dates:
Shakespeare -> from 1996
I -> 1567-1616

Could the sentence be true based on the dates alone: No
----
Sentence: The Dodo was capable of hunting large herbivores like the Iguandodon

Entities and dates:
Dodo -> 1681
Iguanodon -> 99.6 million years ago

Could the sentence be true based on the time periods alone: No
----
Sentence: The engineer upgraded the USS New Ironsides' radar system.

Entities and dates:
USS New Ironsides -> American Civil War
radar -> 1930

Could the sentence be true based on the time periods alone: No
----
Sentence: William the Conqueror enjoyed plenty of chile peppers to flavor his meals.

Entities and dates:
William the Conqueror -> 1028-1087


In [474]:
answer

[' Yes']

### Error analysis

In [530]:
for i in range(len(answers_with_search)):
    print(inputs[i])
    print(answers_with_search[i])
    print(answers_without_search[i])
    print("\nBackground", search_background_1step[i])
    print("---")

William the Conqueror enjoyed plenty of chile peppers to flavor his meals.
['William the Conqueror -> 11th century\nchili peppers -> 20th century\n\nCould the sentence be true based on the time periods alone: No']
['\n\nEntities and dates:\nWilliam the Conqueror -> 11th century\nChile peppers -> 16th century\n\nCould the sentence be true based on the dates alone: No']

Background William I (c. 1028 – 9 September 1087), usually known as William the Conqueror and sometimes William the Bastard, was the first Norman king of England, ...
Red Hot Chili Peppers are an American rock band formed in Los Angeles in 1983, comprising vocalist Anthony Kiedis, bassist Flea, drummer Chad Smith and ...

---
The woman and the wooly mammoth happily played together in the field.
['woman -> anytime\nwooly mammoth -> extinction in the Holocene epoch\n\nCould the sentence be true based on the time periods alone: No']
['\n\nEntities and dates:\nThe woman -> 21st century\nThe wooly mammoth -> 10,000 BC\n\nCoul